In [3]:
import argparse
import numpy as np
import tensorflow as tf
import os
import requests


import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
tf.compat.v1.disable_eager_execution()
# Network Parameters
seed = 123
input_nodes = 101
output_nodes = 101
learning_rate = 0.005
num_iterations = 30
batch_size = 100
weights_folder = '/weights/'

In [5]:
# data fetching
dataSet = 'dataset/nyc_taxi'
filePath = dataSet+'.csv'
df = pd.read_csv(filePath, header=0, skiprows=[1,2], names=['time', 'data', 'timeofday', 'dayofweek'])
df.head(5)

,time,data,timeofday,dayofweek
0,2014-07-01 00:00:00,10844,0,1
1,2014-07-01 00:30:00,8127,30,1
2,2014-07-01 01:00:00,6210,60,1
3,2014-07-01 01:30:00,4656,90,1
4,2014-07-01 02:00:00,3820,120,1


In [6]:
numLags = 100
predictionStep = 3

In [7]:
# standardize data by subtracting mean and dividing by std
meanSeq = np.mean(df['data'])
stdSeq = np.std(df['data'])
df['data'] = (df['data'] - meanSeq)/stdSeq

In [8]:
def getTimeEmbeddedMatrix(sequence, numLags=100, predictionStep=1):
  print("generate time embedded matrix")
  inDim = numLags
  X_train = np.zeros(shape=(len(sequence), inDim))
  y_train = np.zeros(shape=(len(sequence), 1))
  for i in range(numLags-1, len(sequence)-predictionStep):
    X_train[i, :] = np.array(sequence['data'][(i-numLags+1):(i+1)])
    y_train[i, :] = sequence['data'][i+predictionStep]
  print('input shape: ',X_train.shape)
  print('target shape: ',y_train.shape)
  return (X_train, y_train)

In [9]:
X, y = getTimeEmbeddedMatrix(df, numLags, predictionStep)

generate time embedded matrix
input shape:  (17520, 100)
target shape:  (17520, 1)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
train = (X_train,y_train)
test = (X_test,y_test)

(14016, 100) (14016, 1) (3504, 100) (3504, 1)


In [26]:
num_iterations_ = 5
hidden_unit = 100
model = 'GRU' 
print("> Running for", num_iterations_,"iterations")
print("> Hidden size unit", hidden_unit)

> Running for 5 iterations
> Hidden size unit 100


In [27]:
class GRU_cell(object):

    def __init__(self, input_nodes, hidden_unit, output_nodes):

        self.input_nodes = input_nodes
        self.hidden_unit = hidden_unit
        self.output_nodes = output_nodes
        self.Wx = tf.Variable(tf.zeros([self.input_nodes, self.hidden_unit]))

        self.Wr = tf.Variable(tf.zeros([self.input_nodes, self.hidden_unit]))
        self.br = tf.Variable(tf.compat.v1.truncated_normal([self.hidden_unit], mean=1))
        
        self.Wz = tf.Variable(tf.zeros([self.input_nodes, self.hidden_unit]))
        self.bz = tf.Variable(tf.compat.v1.truncated_normal([self.hidden_unit], mean=1))

        self.Wh = tf.Variable(tf.zeros([self.hidden_unit, self.hidden_unit]))

        self.Wo = tf.Variable(tf.compat.v1.truncated_normal([self.hidden_unit, self.output_nodes], mean=1, stddev=.01))
        self.bo = tf.Variable(tf.compat.v1.truncated_normal([self.output_nodes], mean=1, stddev=.01))

        self._inputs = tf.compat.v1.placeholder(tf.float32,shape=[self.input_nodes,self.hidden_unit], name='inputs')
        batch_input_ = tf.transpose(self._inputs, perm=[ 0, 1])
        self.processed_input = tf.transpose(batch_input_)
        self.initial_hidden = self._inputs
        self.initial_hidden = tf.matmul(tf.transpose(self.initial_hidden), tf.zeros([input_nodes, hidden_unit])) 

    def Gru(self, previous_hidden_state, x):
        x = self._inputs
        print("x shape",x.shape)
        print(previous_hidden_state , 'hidden state')
        z = tf.sigmoid(tf.matmul(tf.transpose(x), self.Wz) + self.bz)
        r = tf.sigmoid(tf.matmul(tf.transpose(x), self.Wr) + self.br)
        h_ = tf.tanh(tf.matmul(tf.transpose(x), self.Wx) +
                     tf.matmul(previous_hidden_state, self.Wh) * r)

        current_hidden_state = tf.multiply( (1 - z), h_) + tf.multiply(previous_hidden_state, z)

        return current_hidden_state

    def get_states(self):
        all_hidden_states = tf.scan(self.Gru, self.processed_input, initializer=self.initial_hidden, name='states')
        return all_hidden_states

    def get_output(self, hidden_state):
        print("inside output")
        output = tf.nn.relu(tf.matmul(hidden_state, self.Wo) + self.bo)
        return output

    def get_outputs(self):
        all_hidden_states = self.get_states()
        all_outputs = tf.map_fn(self.get_output, all_hidden_states)
        return all_outputs

In [28]:
def create_batches(x, y, batch_size=101):
    perm = np.random.permutation(len(x))
    x = x[perm]
    y = y[perm]

    batch_x = np.array_split(x, len(x) / batch_size)
    batch_y = np.array_split(y, len(y) / batch_size)
    
    return batch_x, batch_y

In [43]:
def SGD(train, test, hidden_unit, alpha=learning_rate, isTrain=False, num_iterations=num_iterations, batch_size=100):
    np.random.seed(seed)
    tf.compat.v1.set_random_seed(seed)
    (trainX, trainY) = train
    (testX, testY) = test
    (n_x, m) = trainX.T.shape

    Y = tf.compat.v1.placeholder(tf.float32, shape=[None, output_nodes], name='inputs')
    print("Calling GRU cell")
    rnn = GRU_cell(input_nodes, hidden_unit, output_nodes)
    print("Calling get outputs")
    outputs = rnn.get_outputs()
    print("outside get output")
    prediction = tf.nn.softmax(outputs[-1])

    cost = -tf.reduce_sum(Y * tf.compat.v1.log(prediction))
    saver = tf.compat.v1.train.Saver(max_to_keep=10)

    optimizer = tf.compat.v1.train.GradientDescentOptimizer(alpha).minimize(cost)
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32))) * 100

    init = tf.compat.v1.global_variables_initializer()
    sess = tf.compat.v1.Session()
    sess.run(init)

    if not os.path.isdir(os.getcwd() + weights_folder):
        print('Missing folder made')
        os.makedirs(os.getcwd() + weights_folder)

    if isTrain:
        num_minibatches = len(trainX) / batch_size
        for iteration in range(num_iterations):
            iter_cost = 0.
            print(trainX.shape,trainY.shape,'x,y shape')
            (batch_x, batch_y) = create_batches(trainX, trainY, batch_size=batch_size)
            
            for (minibatch_X, minibatch_Y) in zip(batch_x, batch_y):
                print(minibatch_X.shape,minibatch_Y.shape)
              #  minibatch_X = minibatch_X.reshape(1,10100 )
                minibatch_Y = minibatch_Y.reshape((-1,1,101))
                print(minibatch_X.shape,rnn._inputs.shape)
                _, minibatch_cost, acc = sess.run([optimizer, cost, accuracy], feed_dict={rnn._inputs: minibatch_X, Y: minibatch_Y})
                iter_cost += minibatch_cost*1.0 / num_minibatches

            print("Iteration {iter_num}, Cost: {cost}, Accuracy: {accuracy}".format(iter_num=iteration, cost=iter_cost, accuracy=acc))

        # print ppretty(rnn)
        Train_accuracy = str(sess.run(accuracy, feed_dict={rnn._inputs: trainX, Y: trainY}))
        # Test_accuracy = str(sess.run(accuracy, feed_dict={rnn._inputs: testX, Y: testY}))

        save_path = saver.save(sess, "." + weights_folder + "model_" + model + "_" + str(hidden_unit) + ".ckpt")
        print("Parameters have been trained and saved!")
        print("\rTrain Accuracy: %s" % (Train_accuracy))

    else:  # test mode
        # no need to download weights in this assignment
        # check_download_weights(model, hidden_unit)

        saver.restore(sess, "." + weights_folder + "model_" + model + "_" + str(hidden_unit) + ".ckpt")
        acc = sess.run(accuracy, feed_dict={rnn._inputs: testX, Y: testY})
        print("Test Accuracy:"+"{:.3f}".format(acc))

    sess.close()


In [44]:
 SGD(train, test, isTrain=True, num_iterations=num_iterations_, hidden_unit = hidden_unit)

Calling GRU cell
Calling get outputs
x shape (101, 100)
Tensor("placeholder_1:0", shape=(100, 100), dtype=float32) hidden state
inside output
outside get output
(14016, 100) (14016, 1) x,y shape
(101, 100) (101, 1)
(101, 100) (101, 100)


ValueError: Cannot feed value of shape (1, 1, 101) for Tensor 'inputs_20:0', which has shape '(None, 101)'